In [251]:
import os
import glob
import json
import csv
from collections import defaultdict
import pandas as pd
import sys
import argparse
import re
from itertools import permutations

In [312]:
class EvaluateT1k:
    
    def __init__(self, inpath = '', gene = 'HLA', 
                 gtrue = '', outpath = '', method = 'T1K', mode = ''):
    
        '''
        gene = HLA, KIR
        method: T1K, arcasHLA, Optitype
        mode: ingnore
        '''
        self.path = inpath
        self.gene = gene
        self.gtrue = gtrue
        self.gtruth = {}
        self.samples = []
        self.outpath = outpath
        self.qualityThreshold = 0
        self.method = method
        self.mode = mode
        self.total_sum = defaultdict(list)
        self.samplesOut = defaultdict(list)
        
        if gene == 'HLA':
            self.header = ['sample', 'HLA-A', 'HLA-B','HLA-C', 'HLA-DRB1', 'HLA-DQB1']
        
        elif gene == 'KIR':
            self.header = ['sample', 'KIR3DL2']
        
    def readDirs(self):
        print("Initializing current directories...")
        for file in os.listdir(self.path):
            fpath = self.path + file
            if os.path.isdir(fpath):
                self.samples.append(file)
        
        print("Found ", len(self.samples), " number of sample output!", )
    
    def setGtruth(self):
        print("Reformatting and Storing ground truth...")
        with open(self.gtrue, 'r') as gt:
            for line in gt:
                cols = line.rstrip().split()
                sname = cols[0]
                self.gtruth[sname] = {}
                for i in range(1, len(self.header)):
                    gene = self.header[i]
                    
                    # missing the ground truth of this gene                    
                    if len(cols) <= i:
                         self.gtruth[sname][gene] = []
                    
                    else:
                        #complete allele names are constructed
                        alleles = cols[i].split('/')
                        self.gtruth[sname][gene] = ['.']*len(alleles)
                        #Ignore 003 mode
                        if self.mode == 'ignore':
                            if '003' in alleles:
                                #print('003 found in sample')
                                self.gtruth[sname][gene] = []
                            else:
                                nalleles = [gene + "*" + a for a in alleles]
                                self.gtruth[sname][gene] = nalleles
                        elif 'DL1' in alleles:
                            #print('DL1 found in sample')
                            self.gtruth[sname][gene] = []
                        else:
                            nalleles = [gene + "*" + a for a in alleles]
                            self.gtruth[sname][gene] = nalleles
    
    def ParseAlleleName(self, n):
        cols = n.split("*")
        if (self.gene == "HLA"):
            subcols = cols[1].split(":")
            return cols[0], cols[0] + "*" + ":".join(subcols[0:3]), n
        else:
            return cols[0], cols[0] + "*" + cols[1][0:3], n
        
    def getMaxAllele(self, genotype, tgenotype):
        '''
        genotype = ['001', '003']
        tgenotype = ['001', '002', '003']
        '''

        maxScore = 0
        maxAllele = ''
        for i in permutations(tgenotype,2):
            alleleP = [a for a in i]
            score = sum([1 if (genotype[r] == alleleP[r]) else 0 for r in range(2)])
            if score >= maxScore:
                maxScore = score
                maxAllele = alleleP
        
        return maxAllele
    
    def matchAlleles(self):
        
        print("Start matching alleles...")
        for sname in self.samples:
            genotype = {}
            allGenes = {}
            if self.method == "T1K":
                fgenotype = os.path.join(self.path, sname,sname) + '_genotype.tsv'
            elif self.method == "arcasHLA":
                fgenotype = os.path.join(self.path, sname,sname) + '.genotype.json'
            elif self.method == "Optitype":
                fgenotype = os.path.join(self.path, sname,sname) + '_result.tsv'
            if sname not in self.gtruth:
                continue
            else:
                tgenotype = self.gtruth[sname]
            
            with open(fgenotype, 'r') as fh:
                for line in fh:
                    cols = line.rstrip().split()
                    if self.method == 'T1K':
                        gene = cols[0]
                        if (gene not in genotype):
                            genotype[gene] = [".", "."]
                        if (gene not in allGenes):
                            allGenes[gene] = 1
                        
                        allele_1 = "."
                        if (len(cols) >= 5):
                            allele_1_s = cols[2].split(":")[0:2]
                            allele_1_n = ":".join(allele_1_s)
                            allele_1 = re.sub('[A-Za-z]+$', "", allele_1_n)
                            if (float(cols[4]) > 0):
                                genotype[gene][0] = allele_1

                        if (len(cols) >= 8):
                            allele_2_s = cols[5].split(":")[0:2]
                            allele_2_n = ":".join(allele_2_s)
                            allele_2 = re.sub('[A-Za-z]+$', "", allele_2_n)
                            if (float(cols[7]) > 0):
                                genotype[gene][1] = allele_2
                            if (float(cols[7]) == -1):
                                genotype[gene][1] = allele_1
                        else:
                            genotype[gene][1] = allele_1
                    
                    elif self.method == 'arcasHLA':
                        patterns = re.findall('\".*?\"', line)
                        for p in patterns:
                            if "*" not in p:
                                continue
                            idx = 1
                            allele = p[1:-1]
                            n = str(self.gene) + "-" + allele
                            gene, majorAllele, allele = self.ParseAlleleName(n)
                            if (gene not in genotype):
                                genotype[gene] = [".", "."]
                                idx = 0
                            if (gene not in allGenes):
                                allGenes[gene] = 1
                            allele_1_s = majorAllele.split(":")[0:2]
                            allele_1_n = ":".join(allele_1_s)
                            allele_1 = re.sub('[A-Za-z]+$', "", allele_1_n)
                            genotype[gene][idx] = allele_1
                            if genotype[gene][1] == '.':
                                genotype[gene][1] = genotype[gene][0]
                    
                    elif self.method == 'Optitype':
                        if 'Reads' not in cols:
                            genes = ["HLA-A", "HLA-B", "HLA-C"]
                            for g in genes:
                                if (g not in genotype):
                                    genotype[g] = [".", "."]
                                if (g not in allGenes):
                                    allGenes[g] = 1

                            strG = 'HLA-'
                            idx = 0
                            for i in range(1,7):
                                allele = strG + cols[i]
                                gene = genes[idx]
                                if i%2 == 1:
                                    genotype[gene][0] = allele
                                else:
                                    genotype[gene][1] = allele
                                    idx +=1
            for gene in sorted(allGenes.keys()):
                matchCnt = [0, 0, 0, 0]
                if gene not in tgenotype:
                    continue
                elif tgenotype[gene] == []:
                    continue
                    #print(gene, " not found in sample ", sname)
                else:
                    maxAllele = self.getMaxAllele(genotype[gene], tgenotype[gene])
                    
                    for k in range(2):
                        if (genotype[gene][k] == maxAllele[k]):
                            matchCnt[0] += 1
                        else:
                            matchCnt[2] += 1

                    accuracy = 1
                    weakAccuracy = 1
                    total = sum(matchCnt[0:3])
                    if (total > 0):
                        accuracy = matchCnt[0] / total
                        weakAccuracy = (matchCnt[0] + matchCnt[1]) / total
                    s = "\t".join([gene] + ["%.4f"%accuracy, "%.4f"%weakAccuracy] + [str(x) for x in matchCnt]) + "\n"
                    self.total_sum[gene].append(s)
                    self.samplesOut[gene].append(sname)
    
    def writeOut(self):
        
        print("Start writing output file...")
        file = self.outpath + "allele_level_summary.tsv"
        os.makedirs(os.path.dirname(self.outpath), exist_ok=True)
        ppath = self.outpath + "result_sum/"
        os.makedirs(os.path.dirname(ppath), exist_ok=True)
        
        print("Final output:")
        with open(file, "w") as outf:
            for gene in self.total_sum.keys():
                sample = self.samplesOut[gene]
                match = 0
                partmatch = 0
                mismatch = 0
                ignore = 0
                strongMatch = 1
                weakMatch = 1
                index = 0
                ffile = ppath + gene + "_consistency.tsv"
                with open(ffile, "w") as out:
                    for line in self.total_sum[gene]:
                        s = sample[index]
                        out.write("\t".join([s,line]))
                        index +=1
                        line = line.strip().split("\t")
                        match += int(line[3])
                        partmatch += int(line[4])
                        mismatch += int(line[5])
                        ignore += int(line[6])
                    total = match + partmatch + mismatch
                    if (total > 0):
                        strongMatch = match / total
                        weakMatch = (match + partmatch) / total
                    finalOut = "\t".join([gene, "%.4f"%strongMatch, "%.4f"%weakMatch , str(match), str(partmatch), str(mismatch), str(ignore)]) + "\n"
                    outf.write(finalOut)
                    print(finalOut)

### t1k hla rna

In [262]:
groundtruth = 'HLA_ground_truth.tsv'
inPath = './'
outPath = './'
t1kResult = EvaluateT1k(inpath = inPath, gtrue=groundtruth, outpath=outPath)
t1kResult.readDirs()
t1kResult.setGtruth()
t1kResult.matchAlleles()
t1kResult.writeOut()

Initializing current directories...
Found  463  number of sample output!
Reformatting and Storing ground truth...
Start matching alleles...
Start writing output file...
Final output:
HLA-A	0.9957	0.9957	922	0	4	0

HLA-B	0.9946	0.9946	921	0	5	0

HLA-C	0.9892	0.9892	916	0	10	0

HLA-DQB1	0.9888	0.9888	706	0	8	0

HLA-DRB1	0.9903	0.9903	917	0	9	0



### t1k hla wes

In [341]:
groundtruth = 'HLA_ground_truth.tsv'
inPath = './'
outPath = './'
t1kResult = EvaluateT1k(inpath = inPath, gtrue=groundtruth, outpath=outPath)
t1kResult.readDirs()
t1kResult.setGtruth()
t1kResult.matchAlleles()
t1kResult.writeOut()

Initializing current directories...
Found  461  number of sample output!
Reformatting and Storing ground truth...
Start matching alleles...
Start writing output file...
Final output:
HLA-A	0.9425	0.9425	869	0	53	0

HLA-B	0.9805	0.9805	904	0	18	0

HLA-C	0.9610	0.9610	886	0	36	0

HLA-DQB1	0.9577	0.9577	680	0	30	0

HLA-DRB1	0.9848	0.9848	908	0	14	0



### t1k hla wes whitelist

In [339]:
groundtruth = 'HLA_ground_truth.tsv'
inPath = './'
outPath = './'
t1kResult = EvaluateT1k(inpath = inPath, gtrue=groundtruth, outpath=outPath)
t1kResult.readDirs()
t1kResult.setGtruth()
t1kResult.matchAlleles()
t1kResult.writeOut()

Initializing current directories...
Found  461  number of sample output!
Reformatting and Storing ground truth...
Start matching alleles...
Start writing output file...
Final output:
HLA-A	0.9816	0.9816	905	0	17	0

HLA-B	0.9870	0.9870	910	0	12	0

HLA-C	0.9805	0.9805	904	0	18	0

HLA-DQB1	0.0000	0.0000	0	0	710	0

HLA-DRB1	0.0000	0.0000	0	0	922	0



### arcasHLA rna

In [265]:
groundtruth = 'HLA_ground_truth.tsv'
inPath = './'
outPath = './'
t1kResult = EvaluateT1k(inpath = inPath, gtrue=groundtruth, outpath=outPath, method='arcasHLA')
t1kResult.readDirs()
t1kResult.setGtruth()
t1kResult.matchAlleles()
t1kResult.writeOut()

Initializing current directories...
Found  463  number of sample output!
Reformatting and Storing ground truth...
Start matching alleles...
Start writing output file...
Final output:
HLA-A	0.9309	0.9309	862	0	64	0

HLA-B	0.9924	0.9924	919	0	7	0

HLA-C	0.9773	0.9773	905	0	21	0

HLA-DQB1	0.9468	0.9468	676	0	38	0

HLA-DRB1	0.9924	0.9924	919	0	7	0



### optitype rna 

In [315]:
groundtruth = 'HLA_ground_truth.tsv'
inPath = './'
outPath = './'
t1kResult = EvaluateT1k(inpath = inPath, gtrue=groundtruth, outpath=outPath, gene = 'HLA', method = 'Optitype')
t1kResult.readDirs()
t1kResult.setGtruth()
t1kResult.matchAlleles()
t1kResult.writeOut()

Initializing current directories...
Found  463  number of sample output!
Reformatting and Storing ground truth...
Start matching alleles...
Start writing output file...
Final output:
HLA-A	0.9914	0.9914	918	0	8	0

HLA-B	0.9892	0.9892	916	0	10	0

HLA-C	0.9687	0.9687	897	0	29	0



### optitype wes

In [317]:
groundtruth = 'HLA_ground_truth.tsv'
inPath = './'
outPath = './'
t1kResult = EvaluateT1k(inpath = inPath, gtrue=groundtruth, outpath=outPath, gene = 'HLA', method = 'Optitype')
t1kResult.readDirs()
t1kResult.setGtruth()
t1kResult.matchAlleles()
t1kResult.writeOut()

Initializing current directories...
Found  462  number of sample output!
Reformatting and Storing ground truth...
Start matching alleles...
Start writing output file...
Final output:
HLA-A	0.9913	0.9913	914	0	8	0

HLA-B	0.9870	0.9870	910	0	12	0

HLA-C	0.9620	0.9620	887	0	35	0

